In [ ]:
from libs import cloudflare_r2_service
from tqdm import tqdm
from tqdm.asyncio import tqdm as tqdm_asyncio
import os
import json

IMAGES_FOLDER = "2025-07-06_02-45-54"

In [2]:
working_dir = os.path.join("tmp", IMAGES_FOLDER)
with open(os.path.join(working_dir, "image_data.json"), "r") as f:
    image_data: list[dict] = json.load(f)
image_urls: list[str | None] = []
for data in tqdm(image_data):
    image_urls.append(
        cloudflare_r2_service.upload_file_to_r2(
            file_path=os.path.join(working_dir, f"{data["index"]}.webp"),
            path_prefix=IMAGES_FOLDER,
        )
    )

100%|██████████| 1296/1296 [12:11<00:00,  1.77it/s]


In [3]:
cloudflare_r2_service.remove_dir_in_r2(IMAGES_FOLDER)

100%|██████████| 296/296 [01:32<00:00,  3.21it/s]

已删除 R2 中的路径前缀：2025-07-06_02-45-54


In [3]:
image_urls

['https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/0.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/1.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/2.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/3.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/4.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/5.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/6.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/7.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/8.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/9.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/10.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/11.webp',
 'https://sd-style-images.wall-breaker-no4.xyz/2025-07-06_02-45-54/12.webp',
 'https:/